# EchoPro Python: Minimal workflow <a class="tocSkip">

# Import libraries and configure the Jupyter notebook

In [ ]:
# libraries used in the Notebook
import matplotlib.pyplot as plt
import numpy as np

# Python version of EchoPro
from EchoPro import EchoPro

# Allows us to grab the SemiVariogram class so we can use its models
from EchoPro.semivariogram import SemiVariogram

# Allows us to easily use matplotlib widgets in our Notebook
%matplotlib widget

# Set up  EchoPro for a specific survey year

## Initialize EchoPro object  using configuration files

* `initialization_config.yml` -- parameters independent of survey year
* `survey_year_2019_config.yml` -- parameters specific to survey year
* `source` -- Define the region of data to use e.g. US, CAN, US & CAN
* `exclude_age1` -- States whether age 1 hake should be included in analysis.

In [ ]:
epro_2019 = EchoPro(
    init_file_path='./config_files/initialization_config.yml',
    survey_year_file_path='./config_files/survey_year_2019_config.yml',
    source=3,
    exclude_age1=True
)

## Load and process data 

* Below we grab only the data needed to compute the biomass density
    * This data is stored in `epro_2019`

In [ ]:
%%time
epro_2019.load_data(file_types='all')

In [ ]:
epro_2019.specimen_df.head()

## Compute the normalized biomass density
* The biomass density is stored in `epro_2019`

In [ ]:
%%time
epro_2019.compute_biomass_density()

In [ ]:
epro_2019.final_biomass_table.head()

# Jolly-Hampton CV Analysis

* Compute the mean of the Jolly-Hampton CV value 
    * Performed on data that has not been Kriged

`Note: the algorithm used to compute this value is random in nature.`

In [ ]:
%%time
lat_INPFC = [np.NINF, 36, 40.5, 43.000, 45.7667, 48.5, 55.0000]  # INPFC bounds
CV_JH_mean = epro_2019.run_cv_analysis(lat_INPFC, kriged_data=False)

In [ ]:
print(f"Mean Jolly-Hapmton CV: {CV_JH_mean:.4f}")

# Obtain Kriging Mesh Data

## Access Kriging mesh object
* Reads mesh data files specified by `epro_2019` 

In [ ]:
krig_mesh = epro_2019.get_kriging_mesh()

## Plot the Transects and smoothed contour

* Transect points are represented by a changing color gradient
* The smoothed contour points (200m isobath) are blue 

In [ ]:
# Plot the transect points on a folium map
fmap = krig_mesh.plot_points(epro_2019.final_biomass_table, 
                             cmap_column='Transect', color='hex')

# Plot smoothed contour points 
fmap = krig_mesh.plot_points(krig_mesh.smoothed_contour_gdf, 
                             fmap=fmap, color='blue')

# display the folium map
fmap

## Plot the full mesh

In [ ]:
fmap = krig_mesh.plot_points(krig_mesh.mesh_gdf, color='red')

# display the folium map
fmap

## Apply coordinate transformations
* Longitude transformation
* Lat/Lon to distance

### Transect points

In [ ]:
krig_mesh.apply_longitude_distance_transform(
    epro_2019.final_biomass_table, 
    transform_type='transect'
)

### Mesh points

In [ ]:
krig_mesh.apply_longitude_distance_transform(
    krig_mesh.mesh_gdf, 
    transform_type='mesh'
)

In [ ]:
# plot the transformed mesh points 
plt.plot(krig_mesh.mesh_transf_df.x_mesh, 
         krig_mesh.mesh_transf_df.y_mesh, 'r*', markersize=1.25)
plt.show()

# Compute Semi-Variogram and fit a model

* Compute the normalized semi-variogram 
    * using the normalized biomass density
* Fit a model to the semi-variogram values

## Compute the semi-variogram

### Initialize semi-variogram calculation
* Transformed transect points
* parameters specific to semi-variogram algorithm

In [ ]:
semi_vario = epro_2019.get_semi_variogram(
    krig_mesh,
    params=dict(nlag=30, lag_res=0.002)
)

### Compute the normalized semi-variogram

In [ ]:
%%time
semi_vario.calculate_semi_variogram()

## Fit a model to the semi-variogram

* A widget to easily fit the model

In [ ]:
semi_vario.view_semi_variogram()

# Perform Kriging of biomass density

* Ordinary Kriging 
    * transformed mesh points
    * semi-variogram model
    * normalized biomass density.   

## Initialize Kriging routine

In [ ]:
kriging_params = dict(
    # kriging parameters
    k_max=10,
    k_min=3,
    R=0.0226287,
    ratio=0.001,
    
    # parameters for semi-variogram model
    s_v_params={'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
                'exp_pow': 1.5, 'ls_hole_eff': 0.0},
    
    # grab appropriate semi-variogram model
    s_v_model=SemiVariogram.generalized_exp_bessel
)

# initalize kriging routine
krig = epro_2019.get_kriging(kriging_params)

## Perform Ordinary Kriging

In [ ]:
%%time
mesh_krigbiomass_gdf = krig.run_biomass_kriging(krig_mesh)

In [ ]:
print(f"Total Kriged Biomass Estimate: {mesh_krigbiomass_gdf.krig_biomass_vals.sum():.2f} (kmt)")

### Plot Kriged Biomass estimate in kmt

In [ ]:
# only plot non-zero mesh points 
# mesh_krigbiomass_gt0_gdf = mesh_krigbiomass_gdf[mesh_krigbiomass_gdf["krig_biomass_vals"] > 0]
# krig.plot_kriging_results(mesh_krigbiomass_gt0_gdf, krig_fieldname="krig_biomass_vals")

# plot all mesh points
krig.plot_kriging_results(mesh_krigbiomass_gdf, krig_fieldname="krig_biomass_vals")